In [39]:
pip install -Uq pymorphy2 nltk wordcloud matplotlib pandas plotly spacy 

Note: you may need to restart the kernel to use updated packages.


In [1]:
import spacy
nlp = spacy.load('ru_core_news_sm')
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer

stop = stopwords.words("russian")

In [2]:
tivect = TfidfVectorizer()

In [40]:
class DIYPreProcessor:

    def pre_process(self, text: str) -> list:
        doc = nlp(text)
        return [token.lemma_ for token in doc if not token.is_punct and not token.is_stop]

    def tf(self, word: str, text: list) -> float:
        return text.count(word)/len(text) 

    def idf(self, word: str, collection: list) -> float:
        try:
            return np.log(len(collection)/len([doc for doc in collection if word in doc]))
        except ZeroDivisionError as e:
            print(f"Слово {word} не найдено ни в одном документе из корпуса!")
            return  

    def word_to_vector(self, word: str, text: list, collection: list) -> float:        
        try:
            new_collection = []
            for doc in collection:
                new_collection.append(self.pre_process(doc))
            return self.tf(word, self.pre_process(text))*self.idf(word, new_collection)
        except TypeError as e:
            return "Такого слова в корпусе не найдено!"

    def similarity(self, text_a: list, text_b: list, corpus: list) -> float:
        def sparse_row_norm(matrix):        # Обходной путь, т.к. нампай не работает с разреженными матрицами
            out = np.zeros(matrix.shape[0])
            nz, = np.diff(matrix.indptr).nonzero()
            out[nz] = np.sqrt(np.add.reduceat(np.square(matrix.data),matrix.indptr[nz]))
            return out
        text_a, text_b = tivect.fit_transform([text_a, text_b])
        dproduct = text_a.dot(text_b.T)
        norms = sparse_row_norm(text_a)*sparse_row_norm(text_b) 
        return f"{(dproduct/norms).data}"

    def word_pos(self, text: str, start: int, end: int) -> list:
        return [f'{word.lemma_}_{word.pos_}' for word in nlp(text) if not word.is_punct and not word.is_stop][int(start):int(end)]

In [41]:
dpp = DIYPreProcessor()

In [5]:
files = {
    "./Texts/chudesniy_doktor.txt": "Чудесный Доктор",              # 1897
    "./Texts/nochnaya_smena.txt": "Ночная Смена",                   # 1899
    "./Texts/beliy_pudel.txt": "Белый Пудель",                      # 1903
    "./Texts/poedinok.txt": "Поединок",                             # 1905
    "./Texts/izumrud.txt": "Изумруд",                               # 1907
    "./Texts/granatovy_braslet.txt": "Гранатовый Браслет",          # 1910
    "./Texts/anaphema.txt": "Анафема",                              # 1913
    "./Texts/zvezda_solomona.txt": "Звезда Соломона",               # 1917
    "./Texts/limonnaya_korka.txt": "Лимонная Корка",                # 1920
    "./Texts/sinaya_zvezda.txt": "Синяя Звезда",                    # 1927 
    }
kuprin_vec = {}

for path, name in files.items():
    with open(path, "r") as f:
        kuprin_vec.update({name: f.read()})

In [24]:
f'TF-IDF = {dpp.word_to_vector("старик", kuprin_vec["Белый Пудель"], kuprin_vec.values()):.5}'

'TF-IDF = 0.0026295'

In [ ]:
f'TF-IDF = {dpp.word_to_vector("пачпорт", kuprin_vec["Белый Пудель"], kuprin_vec.values()):.5}'

'TF-IDF = 0.0024513'

In [ ]:
f'TF-IDF = {dpp.word_to_vector("изумруд", kuprin_vec["Изумруд"], kuprin_vec.values()):.5}'

'TF-IDF = 0.056523'

In [ ]:
f'TF-IDF = {dpp.word_to_vector("пассат-б4", kuprin_vec["Поединок"], kuprin_vec.values())}'        # Ну, такого слова в корпусе точно нет)

Слово не найдено ни в одном документе из корпуса!


'TF-IDF = Такого слова в корпусе не найдено!'

In [42]:
dpp.similarity(kuprin_vec["Изумруд"], kuprin_vec["Поединок"], kuprin_vec.values())

'[0.66609455]'

In [43]:
i = 0
for name_a, text_a in kuprin_vec.items():
    for name_b, text_b in list(kuprin_vec.items())[i:]:
        if name_a == name_b: continue
        print(f'Косинусное сходство между "{name_a}" и "{name_b}": {dpp.similarity(text_a, text_b, kuprin_vec.values())}')
    i += 1

Косинусное сходство между "Чудесный Доктор" и "Ночная Смена": [0.58602404]
Косинусное сходство между "Чудесный Доктор" и "Белый Пудель": [0.62972775]
Косинусное сходство между "Чудесный Доктор" и "Поединок": [0.71430667]
Косинусное сходство между "Чудесный Доктор" и "Изумруд": [0.53663521]
Косинусное сходство между "Чудесный Доктор" и "Гранатовый Браслет": [0.66731796]
Косинусное сходство между "Чудесный Доктор" и "Анафема": [0.56525408]
Косинусное сходство между "Чудесный Доктор" и "Звезда Соломона": [0.67618802]
Косинусное сходство между "Чудесный Доктор" и "Лимонная Корка": [0.5177606]
Косинусное сходство между "Чудесный Доктор" и "Синяя Звезда": [0.54261528]
Косинусное сходство между "Ночная Смена" и "Белый Пудель": [0.68438475]
Косинусное сходство между "Ночная Смена" и "Поединок": [0.76373817]
Косинусное сходство между "Ночная Смена" и "Изумруд": [0.58747901]
Косинусное сходство между "Ночная Смена" и "Гранатовый Браслет": [0.71714546]
Косинусное сходство между "Ночная Смена" и "